# Slope Hillshading a DEM

In remote sensing you often have to present classification results as maps and need a nice unobtrusive background that gives the viewer an idea where everything is located without being as distracting as a RGB satellite image.

A nice choice for this is a grey hillshading or slope relief background that adds some texture information to your map by using a Digital Elevation Model with the same or higher resolution than the results you want to show.

## Downloading the DEM

A popular (and free) choice for this would be the **SRTM 1-arc second DEM** which is available for almost all regions throughout the world.

Download can be either done through [EarthExplorer](http://earthexplorer.usgs.gov/) or directly from [http://e4ftl01.cr.usgs.gov/](http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/). The easiest way for multiple tiles is finding out their names on EarthExplorer and then download them from e4ftl01 through wget.

This example will use four tiles placed in the Hanzhong Plain, China.

In [2]:
%%bash
# download
cd /data/China/slope_relief/
wget http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N32E106.SRTMGL1.hgt.zip
wget http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N32E107.SRTMGL1.hgt.zip
wget http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N33E106.SRTMGL1.hgt.zip
wget http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N33E107.SRTMGL1.hgt.zip

--2016-02-18 08:29:15--  http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/N32E106.SRTMGL1.hgt.zip
Resolving e4ftl01.cr.usgs.gov (e4ftl01.cr.usgs.gov)... 152.61.133.5, 2001:49c8:4000:121d::5
Connecting to e4ftl01.cr.usgs.gov (e4ftl01.cr.usgs.gov)|152.61.133.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16194109 (15M) [application/zip]
Saving to: ‘N32E106.SRTMGL1.hgt.zip’

     0K .......... .......... .......... .......... ..........  0% 89.4K 2m56s
    50K .......... .......... .......... .......... ..........  0%  347K 1m51s
   100K .......... .......... .......... .......... ..........  0%  366K 88s
   150K .......... .......... .......... .......... ..........  1% 7.14M 66s
   200K .......... .......... .......... .......... ..........  1%  371K 61s
   250K .......... .......... .......... .......... ..........  1% 7.66M 51s
   300K .......... .......... .......... .......... ..........  2% 17.9M 44s
   350K .......... .......... .......... .......

In [14]:
%%bash
# unzip and remove
unzip /data/China/slope_relief/\*.zip -d /data/China/slope_relief/
rm /data/China/slope_relief/\*.zip

unzip:  cannot find or open /data/China/slope_relief/*.zip, /data/China/slope_relief/*.zip.zip or /data/China/slope_relief/*.zip.ZIP.

No zipfiles found.
rm: cannot remove ‘/data/China/slope_relief/*.zip’: No such file or directory


## Merge

In a next step we merge the .hgt DEMs into a single GeoTiff. This step could also be used for warping into a different projection, resolution, etc.

In [15]:
%%bash
# merge
cd /data/China/slope_relief/
gdal_merge.py -o hanzhong-dem.tif *.hgt

0...10...20...30...40...50...60...70...80...90...100 - done.


## Calculate Slope

For this we will use `gdaldem` with the additional `-s` parameter to account for the difference in horizontal units (degree) and vertical units (meters). Also we want to compute at raster edges and near no_data values and therefore invoke `-compute_edges`. If you want the slope in percent instead of degree use `-p`.

In [16]:
%%bash
cd /data/China/slope_relief/
gdaldem slope -s 111120 -compute_edges hanzhong-dem.tif hanzhong-slope.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


# Coloring

In a last step we want to color our results. `gdaldem color-relief` is usually used to color DEMs in green, brown nad yellow. We will use it to color our slopes from white (0 degrees) to black (90 degrees).

Therefore prepare a `slope_color.txt` with the content:
```
0 255 255 255
90 0 0 0
```

In [17]:
%%bash
cd /data/China/slope_relief/
gdaldem color-relief hanzhong-slope.tif slope_color.txt hanzhong-slope-relief.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
